## import librairies

In [1]:
from compas_vol.analysis import Gradient
from compas.geometry import Point, Sphere
from compas_vol.primitives import VolSphere

In [2]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import add_cube

## create volumetric object

In [3]:
vs = VolSphere(Sphere(Point(30, 30, 30), 25.))

## analyse gradient

In [4]:
g = Gradient(vs)

## workspace (dense grid)

In [5]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -30.0, 30.0
lby, uby = -30.0, 30.0
lbz, ubz = -30.0, 30.0
# resolution(s)
nx, ny, nz = 100, 100, 100
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [6]:
dm = vs.get_distance_numpy(x, y, z)
gm = g.get_gradient_numpy(x, y, z)

In [7]:
print(dm.shape)
print(gm.shape)

(100, 100, 100)
(100, 100, 100, 3)


## generate isosurface (marching cube)

In [8]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx,gy,gz))
v += [lbx,lby,lbz]

## display mesh

In [9]:

gm2d = np.reshape(np.swapaxes(gm,0,1), (x.shape[0]*y.shape[1]*z.shape[2], 3))
pts = np.reshape(np.stack(np.meshgrid(x,y,z), axis=3), (x.shape[0]*y.shape[1]*z.shape[2], 3))

In [10]:
# create factor based on distance matrix
f = np.reshape(dm, (x.shape[0]*y.shape[1]*z.shape[2]))[..., np.newaxis]
f = np.interp(f, (f.min(), f.max()), (-6, 6))

In [11]:
p = mp.plot(np.zeros(1))
p.add_lines(pts, pts-gm2d*f, shading={"line_color": "blue"})
vs,ve = add_cube(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))

/Users/bernham/anaconda3/envs/compasdev/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

2